#    
<h1></h1>
<h1><center>ARQUITECTURA EN BIG DATA</center></h1>
<h1><center>SEGUIMIENTO No.2</center></h1>

Para los dos primeros puntos use un notebook sea de Jupyter o de Colab. Descargue el dataset "genoma_2021.zip" del siguiente link: https://grouplens.org/datasets/book-genome/, considere solo los archivos "ratings.json" y "metadata.json", en la archivo "readme.txt" encontrara información de cada
uno de estos archivos. Una vez descomprima el archivo .zip ubique los archivos en:
- .\book_dataset\raw\ratings.json
- .\book_dataset\raw\metadata.json

## Punto 1
Inicialice Spark y cargue los datos del archivo "ratings.json" a una RDD llamado "ratings". Usando el archivo metadata.json cree un diccionario llamado "titulos", como variable broadcast de spark, que contenga como "key" el id del libro (id_item) y como "valor" una lista con los siguientes elementos:
El titulo (title), los autores (authors), y el año

In [1]:
! pip install findspark pyspark

In [2]:
from google.colab import drive
drive.mount('/content/drive')

from pyspark import SparkConf, SparkContext
import findspark
import json

findspark.init()
conf = SparkConf().setMaster("local[*]").setAppName("Seguimiento2")
sc = SparkContext(conf = conf)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
ratings = sc.textFile('/content/drive/MyDrive/Semestre 9/Arquitectura BigData/Seguimiento 2/book-data/ratings.json')

In [4]:
def built_metadata():
  dictio = {}
  with open('/content/drive/MyDrive/Semestre 9/Arquitectura BigData/Seguimiento 2/book-data/metadata.json') as file:
    for line in file.readlines():
      line = json.loads(line)
      dictio[str(line['item_id'])] = {
          'title' : line['title'],
          'year' :  line['year'],
          'authors': line['authors']
      }
  return dictio

#crear y transmitir el diccionario como variable broadcast
titulos = sc.broadcast(built_metadata())

## Punto 2
Liste el (los) nombre(s) de(l) autor(es) del libro y el nombre del libro con mejor rating, en caso de ser más de un libro listelos todos. Haga lo mismo para el libro con peor rating.

In [5]:
def parseline(x):
  line = json.loads(x)
  return (str(line['item_id']), float(line['rating']))

ratings.map(parseline).take(5)

[('41335427', 5.0),
 ('41335427', 3.0),
 ('41335427', 5.0),
 ('41335427', 5.0),
 ('41335427', 5.0)]

In [6]:
ratings = ratings.map(parseline) #se deja por fuera para ahorrar proceso en el pto 4

ratings.mapValues(lambda x: (x,1))\
       .reduceByKey(lambda x,y: (x[0] + y[0], x[1] + y[1]))\
       .mapValues(lambda x: round(x[0] / x[1],2))\
       .sortBy(lambda x: -x[1])\
       .map(lambda x: (titulos.value[x[0]]['title'], x[1]))\
       .take(5)

[('Lovely Trigger (Tristan & Danika, #3)', 4.83),
 ('The Gravity of Us (Elements, #4)', 4.82),
 ('March: Book Three (March, #3)', 4.8),
 ('Harry Potter Boxset (Harry Potter, #1-7)', 4.79),
 ('Long Way Down (Calloway Sisters, #4; Addicted, #3.2)', 4.79)]

In [10]:
ratings_prom = ratings.mapValues(lambda x: (x, 1))\
                      .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))\
                      .mapValues(lambda x: round(x[0] / x[1], 2))

#   

Para el siguiente ejercicio use Flask y construya una APP que acepte los siguientes ENDPOINTS con sus especificaciones, entregue un archivo llamado app.py

In [7]:
#ngrok
! pip install pyngrok
! ngrok authtoken 2oILEUkg8FP4qNHVEzyLVbnEU0M_6e3GutVah3589eqGbuxLQ

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


## Punto 3

Cree un ENDPOINT llamada “book_by_rating” que acepte dos parámetros llamado "rating_min" y “rating_max”, que acepte dos numero entre 1 y 5, y devuelva listado los títulos y autores de los libros que estén dentro de esta calificación promedio, tenga en cuenta que rating_min siempre tiene que ser
menor a rating_max.

In [12]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
from pyspark import SparkConf, SparkContext
import findspark
import json

app = Flask('Book Ratings')

@app.route('/')
def index():
    return '<h2>Punto 3</h2>'

# Endpoint Pto3: `/book_by_rating`
# /book_by_rating?rating_min=4.2      ->error
# /book_by_rating?rating_min=4.0&rating_max=3.8  ->error
# /book_by_rating?rating_min=3.0&rating_max=4.5

@app.route('/book_by_rating')
def book_by_rating():

  try:
      rating_min = float(request.args.get('rating_min'))
      rating_max = float(request.args.get('rating_max'))
  except TypeError:
      return 'Se requieren ambos parámetros: "rating_min" y "rating_max"'

  if not (1 <= rating_min <= 5) or not (1 <= rating_max <= 5):
      return '"rating_min" y "rating_max" deben estar entre 1 y 5'

  if rating_min >= rating_max:
      return 'Especifique un valor de "rating_max" mayor al valor de "rating_min"'

  results = ratings_prom.filter(lambda x: rating_min <= x[1] <= rating_max)\
                        .map(lambda x: {
                            'title': titulos.value[x[0]]['title'],
                            'authors': titulos.value[x[0]]['authors'],
                            'rating': x[1]})\
                        .collect()
  return results

tunnel_flask = ngrok.connect(5000)
print(f'URL ngrok for Flask: {tunnel_flask.public_url}')

app.run()

URL ngrok for Flask: https://bb16-35-245-132-172.ngrok-free.app
 * Serving Flask app 'Book Ratings'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [15/Nov/2024 19:14:24] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Nov/2024 19:14:24] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [15/Nov/2024 19:14:38] "GET /book_by_rating?rating_min=4.2 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Nov/2024 19:14:49] "GET /book_by_rating?rating_min=4.0&rating_max=3.8 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Nov/2024 19:14:55] "GET /book_by_rating?rating_min=4.0&rating_max=3.8 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Nov/2024 19:15:17] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Nov/2024 19:15:21] "GET /book_by_rating?rating_min=4.0&rating_max=3.8 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Nov/2024 19:16:43] "GET /book_by_rating?rating_min=3.0&rating_max=4.5 HTTP

# Punto 4
Cree un ENDPOINT que se llame “book_by_author” que acepte un parámetro llamado “author” y devuelva una lista con los nombres, el año y la  calificación de los libros del autor del parámetro. Tenga presente que puede existir un libro con más de un autor, si uno de esos autores coincide con el parámetro este libro debería entrar en la lista. También considere hacer la comparación interna (dentro del
ENDPOINT) de los nombres en minúscula y formateando el espacio en blanco con un “+” ejemplo si el autor fuese John Green, el parámetro debe de pasarse como “…?author=John+Green”, el nombre
puede estar en mayúsculas, minúsculas o capitalizado como en este ejemplo

In [16]:
from flask import Flask, request
from pyngrok import ngrok

app = Flask('Book Ratings')

@app.route('/')
def index():
    return '<h2>Punto 4</h2>'

# Endpoint pto4: `/book_by_author`
# /book_by_author?author=fulanito+de+tal
# /book_by_author?author=SCOTT+SNYDER
# /book_by_author?author=John+Green

@app.route('/book_by_author')
def get_book_by_author():
  # Obtener el autor de la URL
  author = request.args.get('author')
  if not author:
    return "El parámetro 'author' es requerido.", 400  # Respuesta si el autor no se pasa

  author = author.replace('+', ' ').lower()  # Reemplazar "+" por espacio y convertir a minúsculas

  # Filtrar los libros cuyo autor coincide con el parámetro
  results = ratings.filter(lambda x: any(a.lower() == author for a in titulos.value[x[0]]['authors'].split(', '))) \
                 .mapValues(lambda x: (x, 1)) \
                 .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])) \
                 .mapValues(lambda x: round(x[0] / x[1], 2)) \
                 .sortBy(lambda x: -x[1]) \
                 .map(lambda x: {
                     'title': titulos.value[x[0]]['title'],
                     'average_rating': x[1]
                 }).collect()  # Recoger todos los resultados

  # Devolver los resultados como JSON
  return jsonify(results)

# Conectar el servidor Flask a través de ngrok
tunnel_flask = ngrok.connect(5000)
print(f'URL ngrok for Flask: {tunnel_flask.public_url}')

app.run()

URL ngrok for Flask: https://a4ae-35-245-132-172.ngrok-free.app
 * Serving Flask app 'Book Ratings'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [15/Nov/2024 19:42:01] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Nov/2024 19:42:01] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [15/Nov/2024 19:42:55] "GET /book_by_author?author=John+Green HTTP/1.1" 200 -


# Punto 5
Cree un ENDPOINT que se llame “book_by_year” que acepte un parámetro llamado “year” y devuelva una lista con los nombres y la calificación de los mejores 10 libros de ese año. Tenga en cuenta que para este ENDPOINT los promedios de los libros deben de ser calculados considerando solo los libros filtrados por el valor del parámetro year (NO DE FORMA GENERAL).

In [17]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
import json

app = Flask('Book Ratings')

@app.route('/')
def index():
    return '<h2>Punto 5</h2>'

# Endpoint pto5: `/book_by_year`
# /book_by_year
# /book_by_year?year=2.002

@app.route('/book_by_year')
def book_by_year():

    year = request.args.get('year')
    if not year:
        return 'Especifique un año en el parámetro “year”, eg: year=2005', 400

    try:
        year = int(year.replace(",", "").replace(".", ""))
    except (TypeError, ValueError):
        return "El parámetro 'year' debe ser un número entero válido.", 400

    # Filtrar libros por el año especificado
    books_filtered_by_year = ratings.filter(lambda x: titulos.value[x[0]]['year'] == year)

    # Calcular el promedio de calificaciones solo para los libros de ese año
    ratings_avg_for_year = books_filtered_by_year.mapValues(lambda x: (x, 1)) \
                                                .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])) \
                                                .mapValues(lambda x: round(x[0] / x[1], 2))  # Promedio de calificación

    # Obtener los 10 mejores libros ordenados por calificación
    top_10_books = ratings_avg_for_year.sortBy(lambda x: -x[1]) \
                                       .map(lambda x: {
                                           'title': titulos.value[x[0]]['title'],
                                           'rating': x[1]
                                       }) \
                                       .take(10)

    if not top_10_books:
        return f"No se encontraron libros para el año {year}.", 404

    return jsonify({'results': top_10_books})

tunnel_flask = ngrok.connect(5000)
print(f'URL ngrok for Flask: {tunnel_flask.public_url}')

app.run()

URL ngrok for Flask: https://89f7-35-245-132-172.ngrok-free.app
 * Serving Flask app 'Book Ratings'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [15/Nov/2024 19:54:36] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Nov/2024 19:54:36] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [15/Nov/2024 19:54:39] "GET /book_by_year HTTP/1.1" 400 -
INFO:werkzeug:127.0.0.1 - - [15/Nov/2024 19:55:26] "GET /book_by_year?year=2.002 HTTP/1.1" 200 -


# Punto 6
Cree un ENDPOINT que se llama “get_book” que acepte los parámetros year, rating_min, rating_max y author; y haga la consulta como se indica en los 3 puntos anteriores considerando todos los parámetros posibles y la ausencia de múltiples de ellos. Tenga en cuenta que este ENDPOINT en especifico tiene distintas variantes según el parámetro y combinación de parámetros que se ingrese, toda combinación debe de tener una salida coherente en dado caso de que la consulta no se pueda hacer.

In [20]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
from pyspark import SparkConf, SparkContext
import findspark
import json

app = Flask('Book Ratings')

@app.route('/')
def index():
    return '<h2>Punto 6</h2>'

# Endpoint pto6: `/get_book`
# /get_book?year=2,002
# /get_book?author=Scott+snyder&year=2002
# /get_book?author=R.K.+Lilley&year=2014
# /get_book?rating_min=4.0&rating_max=3.8
# /get_book?rating_min=4.0&rating_max=5
@app.route('/get_book')
def get_book():

    author = request.args.get('author')
    year = request.args.get('year')
    rating_min = request.args.get('rating_min')
    rating_max = request.args.get('rating_max')

    try:
        if rating_min:
            rating_min = float(rating_min)
        if rating_max:
            rating_max = float(rating_max)
    except ValueError:
        return "Los parámetros 'rating_min' y 'rating_max' deben ser números válidos.", 400

    if (rating_min is not None and rating_max is None) or (rating_max is not None and rating_min is None):
        return "Si se especifica 'rating_min', también se debe especificar 'rating_max' y viceversa.", 400

    if rating_min is not None and rating_max is not None and rating_min > rating_max:
        return "'rating_min' debe ser menor que 'rating_max'.", 400

    filtered_books = ratings_prom

    if author:
        author = author.replace('+', ' ').lower()
        filtered_books = filtered_books.filter(lambda x: any(a.lower() == author for a in titulos.value[x[0]]['authors'].split(', ')))

    if year:
        try:
            cleaned_year = int(year.replace(",", "").replace(".", ""))
            filtered_books = filtered_books.filter(lambda x: titulos.value[x[0]]['year'] == cleaned_year)
        except ValueError:
            return "El parámetro 'year' debe ser un número entero válido.", 400

    if rating_min is not None:
        filtered_books = filtered_books.filter(lambda x: x[1] >= rating_min)
    if rating_max is not None:
        filtered_books = filtered_books.filter(lambda x: x[1] <= rating_max)

    results = filtered_books.mapValues(lambda x: (x, 1))\
                            .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))\
                            .mapValues(lambda x: round(x[0] / x[1], 2))

    sorted_books = results.sortBy(lambda x: -x[1])
    top_books = sorted_books.take(10)

    if not top_books:
        return jsonify({"message": "No se encontraron libros para los filtros especificados."}), 404

    results_list = [{
        'title': titulos.value[book_id]['title'],
        'authors': titulos.value[book_id]['authors'],
        'year': titulos.value[book_id]['year'],
        'rating': rating
    } for book_id, rating in top_books]

    return jsonify(results_list)

tunnel_flask = ngrok.connect(5000)
print(f'URL ngrok for Flask: {tunnel_flask.public_url}')

app.run()

URL ngrok for Flask: https://6770-35-245-132-172.ngrok-free.app
 * Serving Flask app 'Book Ratings'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [15/Nov/2024 21:31:57] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Nov/2024 21:31:57] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [15/Nov/2024 21:32:16] "GET /get_book?rating_min=4.0&rating_max=3.8 HTTP/1.1" 400 -
INFO:werkzeug:127.0.0.1 - - [15/Nov/2024 21:32:29] "GET /get_book?rating_min=4.0&rating_max=5 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Nov/2024 21:33:13] "GET /get_book?author=Scott+snyder&year=2002 HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [15/Nov/2024 21:34:00] "GET /get_book?author=R.K.+Lilley&year=2014 HTTP/1.1" 200 -
